#  FrozenLake
Today you are going to learn how to survive walking over the (virtual) frozen lake through discrete optimization.

<img src="http://vignette2.wikia.nocookie.net/riseoftheguardians/images/4/4c/Jack's_little_sister_on_the_ice.jpg/revision/latest?cb=20141218030206" alt="a random image to attract attention" style="width: 400px;"/>


In [1]:
import gym

#create a single game instance
env = gym.make("FrozenLake-v0")

#start new game
env.reset();

[2017-01-04 11:26:00,051] Making new env: FrozenLake-v0


In [2]:
# display the game state
env.render()

SFFF
FHFH
FFFH
HFFG



### legend

![img](https://cdn-images-1.medium.com/max/800/1*MCjDzR-wfMMkS0rPqXSmKw.png)

### Gym interface

The three main methods of an environment are
* __reset()__ - reset environment to initial state, _return first observation_
* __render()__ - show current environment state (a more colorful version :) )
* __step(a)__ - commit action __a__ and return (new observation, reward, is done, info)
 * _new observation_ - an observation right after commiting the action __a__
 * _reward_ - a number representing your reward for commiting action __a__
 * _is done_ - True if the MDP has just finished, False if still in progress
 * _info_ - some auxilary stuff about what just happened. Ignore it for now

In [4]:
print ("initial observation code:",env.reset())
print ('printing observation:')
env.render()
print ("observations:",env.observation_space, 'n=',env.observation_space.n)
print ("actions:",env.action_space, 'n=',env.action_space.n)

initial observation code: 0
printing observation:
SFFF
FHFH
FFFH
HFFG

observations: Discrete(16) n= 16
actions: Discrete(4) n= 4


In [6]:
print ("taking action 2 (right)")
new_obs, reward, is_done, _ = env.step(2)
print ("new observation code:",new_obs)
print ("reward:", reward)
print ("is game over?:",is_done)
print ("printing new state:")
env.render()

taking action 2 (right)
new observation code: 0
reward: 0.0
is game over?: False
printing new state:
SFFF
FHFH
FFFH
HFFG
  (Right)


In [7]:
action_to_i = {
    'left':0,
    'down':1,
    'right':2,
    'up':3
}

### Play with it
* Try walking 5 steps without falling to the (H)ole
 * Bonus quest - get to the (G)oal
* Sometimes your actions will not be executed properly due to slipping over ice
* If you fall, call __env.reset()__ to restart

In [8]:
env.step(action_to_i['up'])
env.render()

SFFF
FHFH
FFFH
HFFG
  (Up)


### Policy

* The environment has a 4x4 grid of states (16 total), they are indexed from 0 to 15
* From each states there are 4 actions (left,down,right,up), indexed from 0 to 3

We need to define agent's policy of picking actions given states. Since we have only 16 disttinct states and 4 actions, we can just store the action for each state in an array.

This basically means that any array of 16 integers from 0 to 3 makes a policy.

In [9]:
import numpy as np
def get_random_policy():
    """
    Build a numpy array representing agent policy.
    This array must have one element per each of 16 environment states.
    Element must be an integer from 0 to 3, representing action
    to take from that state.
    """
    return np.random.randint(0, 4, 16)

In [10]:
np.random.seed(1234)
policies = [get_random_policy() for i in range(10**4)]

assert all([len(p) == 16 for p in policies]), 'policy length should always be 16'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == 3, 'maximal action id should be 3'
action_probas = np.unique(policies,return_counts=True)[-1] /10**4. /16.
print ("Action frequencies over 10^4 samples:",action_probas)
assert np.allclose(action_probas,[0.25]*4,atol=0.05), "The policies aren't uniformly random (maybe it's just an extremely bad luck)"
print ("Seems fine!")

Action frequencies over 10^4 samples: [ 0.25014375  0.25130625  0.2495375   0.2490125 ]
Seems fine!


### Let's evaluate!
* Implement a simple function that runs one game and returns the total reward

In [11]:
def sample_reward(env,policy,t_max=25):
    """
    Interact with an environment, return sum of all rewards.
    If game doesn't end on t_max (e.g. agent walks into a wall), 
    force end the game and return whatever reward you got so far.
    """
    s = env.reset()
    total_reward = 0
    
    for i in range(t_max):
        s, rev, is_end, _ = env.step(policy[s])
        total_reward += rev
        if is_end: break
    return total_reward

In [12]:
print ("generating 10^3 sessions...")
rewards = [sample_reward(env,get_random_policy()) for _ in range(10**3)]
assert all([type(r) in (int,float) for r in rewards]), 'sample_reward must return a single number'
assert all([0 <= r <= 1 for r in rewards]), 'total rewards should be between 0 and 1 for frozenlake'
print ("Looks good!")

generating 10^3 sessions...
Looks good!


In [85]:
def evaluate(policy,n_times=100):
    """Run several evaluations and average the score the policy gets."""
    rewards = [sample_reward(env, policy) for i in range(n_times)]
    return np.mean(rewards)
        

In [14]:
!pip install tqdm

You are using pip version 8.1.2, however version 9.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [81]:
def print_policy(policy):
    """a function that displays a policy in a human-readable way"""
    lake = "SFFFFHFHFFFHHFFG"
    
    # where to move from each tile
    arrows = ['<v>^'[a] for a in policy]
    
    #draw arrows above S and F only
    signs = [arrow if tile in "SF" else tile for arrow,tile in zip(arrows,lake)]
    
    for i in range(0,16,4):
        print (' '.join(signs[i:i+4]))

print( "random policy:")
print_policy(get_random_policy())

random policy:
^ > < v
^ H v H
> > > H
H < > G


### Random search

In [ ]:
best_policy = None
best_score = -float('inf')

from tqdm import tqdm
for i in tqdm(range(10000)):
    policy = get_random_policy()
    score = evaluate(policy)
    if score > best_score:
        best_score = score
        best_policy = policy
        print "New best score:",score
        print "Best policy:"
        print_policy(best_policy)

### Genetic algorithm

In [62]:
import random
def crossover(policy1,policy2,p=0.5):
    a=[]
    for x in range(16):
        a.append(random.choice([0,1]))
        
    return np.choose(a,[policy1,policy2])

In [63]:
crossover(range(16),range(16))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [35]:
np.random.seed(1234)
policies = [crossover(get_random_policy(),get_random_policy()) 
            for i in range(10**4)]

assert all([len(p) == 16 for p in policies]), 'policy length should always be 16'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == 3, 'maximal action id should be 3'
print ("Seems fine!")

Seems fine!


In [19]:

n_epochs = 100 #how many cycles to make
pool_size = 100 #how many policies to maintain
n_crossovers = 50 #how many crossovers to make on each step
n_mutations = 10 #how many mutations to make on each tick


In [79]:
print ("initializing...")
pool = []
for i in range(pool_size):
    pool.append(get_random_policy())
    
pool_scores = []
for i in range(pool_size):
    pool_scores.append(evaluate(pool[i]))


initializing...


In [48]:
assert type(pool) == type(pool_scores) == list
assert len(pool) == len(pool_scores) == pool_size
assert all([type(score) in (float,int) for score in pool_scores])


In [54]:
def mut(seq,n=1):
    for x in range(1):
        seq[random.randint(0,len(seq)-1)]=random.randint(0,3)
    return(seq)
print(mut([1,2,3,4,5,6]))

[1, 3, 3, 4, 5, 6]


In [ ]:
#main loop
for epoch in range(n_epochs):
    print ("Epoch %s:"%epoch)
    
    crossovered = []
    for x in range(n_crossovers):
        crossovered.append(crossover(random.choice(pool),random.choice(pool)))
    mutated =[]
    for x in range(n_mutations):
        mutated.append(mut(random.choice(pool)))
    
    assert type(crossovered) == type(mutated) == list
    
    #evaluate new scores
    new_policies = crossovered + mutated
    new_scores =[]
    for x in new_policies:
        new_scores.append(evaluate(x))
    
    
    assert type(new_scores) is list and len(new_scores) == len(new_policies)
    
    
    #add new sessions to the pool
    pool = pool + new_policies
    pool_scores = []
    for x in pool:
        pool_scores.append(evaluate(x))
    
    #select pool_size best policies
    selected_indices = np.argsort(pool_scores)[-pool_size:]
    
    
    pool = [pool[i] for i in selected_indices]
    pool_scores = [pool_scores[i] for i in selected_indices]
    
    #print the best policy so far (last in ascending score order)

    print("best score:",max(pool_scores))
    print_policy(pool[-1])

Epoch 0:
best score: 0.25
< ^ > <
< H ^ H
^ v v H
H > v G
Epoch 1:
best score: 0.3
< ^ > <
< H ^ H
^ v v H
H > v G
Epoch 2:
best score: 0.31
< ^ > ^
< H v H
^ v v H
H > v G
Epoch 3:
best score: 0.33
< < < <
< H < H
^ > < H
H > v G
Epoch 4:
best score: 0.37
< ^ v ^
< H < H
^ v < H
H > v G
Epoch 5:
best score: 0.37
< ^ > <
< H ^ H
^ v v H
H > v G
Epoch 6:
best score: 0.36
< ^ > >
< H < H
^ v < H
H > v G
Epoch 7:
best score: 0.38
< ^ > v
< H ^ H
^ v v H
H > v G
Epoch 8:
best score: 0.38
< ^ > v
< H ^ H
^ v v H
H > v G
Epoch 9:
best score: 0.39
< > > <
< H v H
^ v v H
H > v G
Epoch 10:


## moar

The parameters of the genetic algorithm aren't optimal, try to find something better. (size, crossovers and mutations)

Try alternative crossover and mutation strategies
* prioritize crossover over higher-scores
* only mutate several random actions from existing policy, not the whole.
* try to select a more diverse pool, not just best scorers

See which combination works best!

In [78]:
len(pool_scores)

820